# 1. Import the required libraries

In [1]:
# Standard code libraries
import os
import platform
import glob

import numpy as np
from numpy.matlib import repmat

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Custom code libraries from ReSurfEMG
from resurfemg.config.config import Config
from resurfemg.data_connector.tmsisdk_lite import Poly5Reader
from resurfemg.data_classes.data_classes import (
VentilatorDataGroup, EmgDataGroup)

%matplotlib widget

## 2. Load the ventilator and sEMG data

In [ ]:
# Identify all recordings available for the selected patient/measurement_date

# Root directory for test data
config = Config()
root_patient_data_directory = \
    config.get_directory('root_patient_data_directory')

if platform.system() == 'Windows':
    path_sep = "\\"
else:
    path_sep = '/'

emg_pattern = os.path.join(root_patient_data_directory, '**/*.Poly5')
emg_and_vent_files = glob.glob(emg_pattern, recursive=True)

emg_files = []
vent_files = []

for file in emg_and_vent_files:
    if 'Draeger' in file:
        vent_files.append(file)
    else:
        emg_files.append(file)

emg_file_chosen = emg_files[6]
vent_file_chosen = vent_files[6]
print("The chosen files are:\n", emg_file_chosen, '\n', vent_file_chosen)

In [3]:
# # Identify all recordings available for the selected patient/measurement_date

# # Root directory for test data
# config = Config()
# root_patient_data_directory = \
#     config.get_directory('test_data')

# if platform.system() == 'Windows':
#     path_sep = "\\"
# else:
#     path_sep = '/'

# emg_pattern = os.path.join(root_patient_data_directory, '**/*.Poly5')
# emg_and_vent_files = glob.glob(emg_pattern, recursive=True)

# emg_files = []
# vent_files = []

# for file in emg_and_vent_files:
#     if 'vent' in file:
#         vent_files.append(file)
#     else:
#         emg_files.append(file)

# emg_file_chosen = emg_files[0]
# vent_file_chosen = vent_files[0]
# print("The chosen files are:\n", emg_file_chosen, '\n', vent_file_chosen)

In [ ]:
# Load the EMG and ventilator data recordings from the selected folders.
data_emg = Poly5Reader(emg_file_chosen)
data_vent = Poly5Reader(vent_file_chosen)
data_emg_samples = data_emg.samples[:data_emg.num_samples]
fs_emg = data_emg.sample_rate
data_vent_samples = data_vent.samples[:data_vent.num_samples]
fs_vent = data_vent.sample_rate

# Define the time series of the EMG and ventilator recordings
y_emg = data_emg_samples[:, 1*fs_emg:61*fs_emg]
y_vent = data_vent_samples[:, 1*fs_vent:61*fs_vent]

# Define the time axes
t_emg = [i/fs_emg for i in range(len(y_emg[0, :]))]
t_vent = [i/fs_vent for i in range(len(y_vent[0, :]))]

In [ ]:
from resurfemg.visualization.visualization import show_my_power_spectrum, show_psd_welch
fig, axis = plt.subplots(figsize=(6, 6), sharex=True)
# show_my_power_spectrum(
#     y_emg[1, :], fs_emg, fs_emg//2, axis_spec=1,
# )

show_psd_welch(y_emg[1, :], fs_emg, fs_emg, axis_spec=1, signal_unit='uV')

In [ ]:
# Store the EMG data in a group of TimeSeries objects
emg_timeseries = EmgDataGroup(
    y_emg,
    fs=fs_emg,
    labels=['ECG', 'EMGdi', 'EMGpara'],
    units=3*['uV'])


# Data is stored in:
# fs        --> emg_timeseries.fs
# labels    --> emg_timeseries.labels
# units     --> emg_timeseries.units
# ECG       --> emg_timeseries.channels[0] = TimeSeries object
# EMGdi     --> emg_timeseries.channels[1: TimeSeries object
#   with:
#   emg_timeseries.channels[0].fs = fs
#   emg_timeseries.channels[0].y_raw = y_emg[0, :]
#   emg_timeseries.channels[0].t_data = time axis data for y_raw
#   emg_timeseries.channels[0].y_clean = None
#   emg_timeseries.channels[0].y_baseline = None
#   emg_timeseries.channels[0].y_baseline = None
#   etc.
# From the labels 'ECG' is automatically detected.

In [ ]:
# Store the ventilator data in a group of TimeSeries objects
vent_timeseries = VentilatorDataGroup(
    y_vent,
    fs=fs_vent,
    labels=['Paw', 'F', 'Vvent'],
    units=['cmH2O', 'L/s', 'L'])

# Data is stored in:
# fs        --> vent_timeseries.fs
# labels    --> vent_timeseries.labels
# units     --> vent_timeseries.units
# ECG       --> vent_timeseries.channels[0: TimeSeries object
# EMGdi     --> vent_timeseries.channels[1] = TimeSeries object
#   with:
#   vent_timeseries.channels[0].fs = fs
#   vent_timeseries.channels[0].y_raw = y_vent[0, :]
#   vent_timeseries.channels[0].t_data = time axis data for y_raw
#   vent_timeseries.channels[0].y_clean = None
#   vent_timeseries.channels[0].y_baseline = None
#   vent_timeseries.channels[0].y_baseline = None
#   etc.
# From the labels 'P_aw', 'F', and 'Vvent' are automatically detected.

- Bandpass hp_cf (2-80) lc 50 - 1000
- ECG elimination:
    - Gating: Filling methods
    - ICA: In vs out
    - Wavelet denoising
    - Template subtraction
- Envelope
    - RMS: Window 50 - 400 ms
    - ARV: Window 50 - 400 ms
- Baseline crossing detection
    - Graßhoff: Baseline percentile (20, 33, 40)
    - CoV baseline (Warnaar 2024)
    - Slopesum baseline
    - Slope extrapolation
- Features:
    - ETP
    - Max to baseline
    - Risetime

# 3. Band-pass filtering

In [ ]:
# # Iterate over high-pass cut-off frequency
# # EMG data
# fig, axis = plt.subplots(nrows=4, ncols=1, figsize=(6, 8), sharex=True)
# axes_emg = axis[3]

# # - Bandpass lc (2-80) hc 50 - 1000
# hp_cfs = [15, 25, 50, 75, 100]
# # hp_cfs = [2]
# colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red', 'tab:orange']
# # colors = ['tab:purple', 'tab:blue', 'tab:cyan', 'tab:olive', 'tab:green',
# #           'tab:red', 'tab:orange']

# for _, (hp_cf, color) in enumerate(zip(hp_cfs, colors)):
#     # emg_timeseries.filter(hp_cf=hp_cf, channel_idxs=[1], order=1)
#     emg_timeseries.filter(hp_cf=hp_cf, channel_idxs=[1])
#     emg_timeseries.gating(channel_idxs=[1])
#     emg_timeseries.envelope(channel_idxs=[1])
#     emg_timeseries.plot_full(
#         channel_idxs=[1],
#         axes=axes_emg,
#         signal_type='env',
#         baseline_bool=False,
#         colors=[color])

# # Ventilator data data
# axes_vent = axis[:3]
# vent_timeseries.plot_full(axes_vent)
# axis[-1].legend([str(hp_cf) for hp_cf in hp_cfs], bbox_to_anchor=(0., -0.6, 1., -.6), loc='lower left',
#                       ncols=5, mode="expand", borderaxespad=0.)
# axis[-1].set_xlabel('t (s)')

# # axis[-1, 0].axis('off')

# axes_vent[-1].set_xlim([0, 8.0])

In [ ]:
# # Iterate over low-pass cut-off frequency

# # fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(6, 8), sharex=True)
# # axes_emg = axis[3]
# # axes_vent = axis[:3]

# # Side plot
# fig = plt.figure(layout="constrained", figsize=(10, 6))

# gs = GridSpec(3, 2, figure=fig)
# axes_emg = fig.add_subplot(gs[:, 1])
# for i in range(3):
#     ax = fig.add_subplot(gs[i, 0], sharex=axes_emg)
# axis = np.array(fig.axes)
# axes_vent = axis[-3:]

# # EMG data
# # - Bandpass lc (2-80) hc 50 - 1000
# lp_cfs = [50, 100, 250, 375, 500]
# # hp_cfs = [2]
# colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red', 'tab:orange']
# # colors = ['tab:purple', 'tab:blue', 'tab:cyan', 'tab:olive', 'tab:green',
# #           'tab:red', 'tab:orange']

# for _, (lp_cf, color) in enumerate(zip(lp_cfs, colors)):
#     # emg_timeseries.filter(lp_cf=lp_cf, channel_idxs=[1], order=1)
#     emg_timeseries.filter(lp_cf=lp_cf, channel_idxs=[1])
#     emg_timeseries.gating(channel_idxs=[1])
#     emg_timeseries.envelope(channel_idxs=[1])
#     emg_timeseries.plot_full(
#         channel_idxs=[1],
#         axes=axes_emg,
#         signal_type='env',
#         baseline_bool=False,
#         colors=[color])
# axes_emg.set_xlabel('t (s)')
# axes_emg.set_ylim([0, 3.0])
# axes_emg.set_title('Low pass cut-off effects')
# axes_emg.legend(
#     [str(lp_cf) for lp_cf in lp_cfs],
#     bbox_to_anchor=(0.01, 0.95, 0.98, 0.95),
#     loc='lower left',
#     ncols=5, 
#     mode="expand", borderaxespad=0.)
# # Ventilator data data
# vent_timeseries.plot_full(axes_vent)
# axis[-1].set_xlabel('t (s)')

# # axis[-1, 0].axis('off')

# axes_vent[-1].set_xlim([0, 8.0])

In [ ]:
# Iterate over filtering order

# # Stacked plot
# fig, axis = plt.subplots(nrows=4, ncols=1, figsize=(6, 8), sharex=True)
# axes_emg = axis[3]

# Side plot
fig = plt.figure(layout="constrained", figsize=(10, 6))

gs = GridSpec(3, 2, figure=fig)
axes_emg = fig.add_subplot(gs[:, 1])
for i in range(3):
    ax = fig.add_subplot(gs[i, 0], sharex=axes_emg)
axis = np.array(fig.axes)
axes_vent = axis[-3:]


# EMG data
# - Bandpass lc (2-80) hc 50 - 1000
orders = [1, 2, 3, 4, 8]
# hp_cfs = [2]
colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red', 'tab:orange']
# colors = ['tab:purple', 'tab:blue', 'tab:cyan', 'tab:olive', 'tab:green',
#           'tab:red', 'tab:orange']

for _, (order, color) in enumerate(zip(orders, colors)):
    emg_timeseries.filter(order=order, channel_idxs=[1],)
    emg_timeseries.gating(channel_idxs=[1])
    emg_timeseries.envelope(channel_idxs=[1])
    emg_timeseries.plot_full(
        channel_idxs=[1],
        axes=axes_emg,
        signal_type='env',
        baseline_bool=False,
        colors=[color])
# bbox_to_anchor=(0., -0.6, 1., -.6),
axes_emg.legend(
    [str(order) for order in orders], 
    bbox_to_anchor=(0., 1.05, 1., 1.05),
    loc='lower left',
    ncols=5, 
    mode="expand",
    borderaxespad=0.)
axes_emg.set_xlabel('t (s)')




# Ventilator data data
# axes_vent = axis[:3]
vent_timeseries.plot_full(axes_vent)
axes_vent[-1].set_xlabel('t (s)')

# axis[-1, 0].axis('off')

axes_vent[-1].set_xlim([0, 8.0])

In [ ]:
# Iterate over low-pass cut-off frequency

# fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(6, 8), sharex=True)
# axes_emg = axis[3]
# axes_vent = axis[:3]

# Side plot
fig = plt.figure(layout="constrained", figsize=(10, 6))

gs = GridSpec(6, 2, figure=fig)
axes_emg_1 = fig.add_subplot(gs[:3, 1])
axes_emg_2 = fig.add_subplot(gs[3:, 1], sharex=axes_emg_1)
for i in range(3):
    ax = fig.add_subplot(gs[i*2:i*2+2, 0], sharex=axes_emg_1)
axis = np.array(fig.axes)
axes_vent = axis[-3:]

# EMG data
# - Bandpass lc (2-80) hc 50 - 1000
lp_cfs = [50, 100, 250, 375, 500]
# hp_cfs = [2]
colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red', 'tab:orange']
# colors = ['tab:purple', 'tab:blue', 'tab:cyan', 'tab:olive', 'tab:green',
#           'tab:red', 'tab:orange']

for _, (lp_cf, color) in enumerate(zip(lp_cfs, colors)):
    # emg_timeseries.filter(lp_cf=lp_cf, channel_idxs=[1], order=1)
    emg_timeseries.filter(lp_cf=lp_cf, channel_idxs=[1])
    emg_timeseries.gating(channel_idxs=[1])
    emg_timeseries.envelope(channel_idxs=[1])
    emg_timeseries.plot_full(
        channel_idxs=[1],
        axes=axes_emg_1,
        signal_type='env',
        baseline_bool=False,
        colors=[color])
axes_emg_1.set_xlabel('t (s)')
axes_emg_1.set_ylim([0, 3.0])
axes_emg_1.set_title('Low pass cut-off effects')
axes_emg_1.legend(
    [str(lp_cf) for lp_cf in lp_cfs],
    bbox_to_anchor=(0.01, 0.90, 0.98, 0.90),
    loc='lower left',
    ncols=5, 
    mode="expand", borderaxespad=0.)


# - Bandpass lc (2-80) hc 50 - 1000
hp_cfs = [15, 25, 50, 75, 100]
# hp_cfs = [2]
colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red', 'tab:orange']
# colors = ['tab:purple', 'tab:blue', 'tab:cyan', 'tab:olive', 'tab:green',
#           'tab:red', 'tab:orange']

for _, (hp_cf, color) in enumerate(zip(hp_cfs, colors)):
    # emg_timeseries.filter(hp_cf=hp_cf, channel_idxs=[1], order=1)
    emg_timeseries.filter(hp_cf=hp_cf, channel_idxs=[1])
    emg_timeseries.gating(channel_idxs=[1])
    emg_timeseries.envelope(channel_idxs=[1])
    emg_timeseries.plot_full(
        channel_idxs=[1],
        axes=axes_emg_2,
        signal_type='env',
        baseline_bool=False,
        colors=[color])
axes_emg_2.set_xlabel('t (s)')
axes_emg_2.set_ylim([0, 3.0])
axes_emg_2.set_title('High pass cut-off effects')
axes_emg_2.legend(
    [str(hp_cf) for hp_cf in hp_cfs],
    bbox_to_anchor=(0.01, 0.90, 0.98, 0.90),
    loc='lower left',
    ncols=5, 
    mode="expand", borderaxespad=0.)

# Ventilator data data
vent_timeseries.plot_full(axes_vent)
axis[-1].set_xlabel('t (s)')

# axis[-1, 0].axis('off')

axes_vent[-1].set_xlim([0, 8.0])

In [12]:
# Filter
emg_timeseries.filter(
    lp_cf=500,
    hp_cf=25,
    order=3,
)

# 4. ECG-removal

In [ ]:
# fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(6, 8), sharex=True)
# axes_emg = axis[3]
# axes_vent = axis[:3]

# Side plot
fig = plt.figure(layout="constrained", figsize=(10, 6))

gs = GridSpec(6, 2, figure=fig)
axes_emg_1 = fig.add_subplot(gs[:3, 1])
axes_emg_2 = fig.add_subplot(gs[3:, 1], sharex=axes_emg_1)
for i in range(3):
    ax = fig.add_subplot(gs[i*2:i*2+2, 0], sharex=axes_emg_1)
axis = np.array(fig.axes)
axes_vent = axis[-3:]

# EMG data
# Iterate over window length
gate_width_t_ms = [50, 100, 200, 300, 400]
# hp_cfs = [2]
colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red', 'tab:orange']
# colors = ['tab:purple', 'tab:blue', 'tab:cyan', 'tab:olive', 'tab:green',
#           'tab:red', 'tab:orange']

for _, (gate_width_t, color) in enumerate(zip(gate_width_t_ms, colors)):
    emg_timeseries.filter(
        lp_cf=500,
        hp_cf=25,
        order=3,
    )
    gate_width_t_s = (gate_width_t *  fs_emg) // 1000
    emg_timeseries.gating(
        channel_idxs=[1], 
        gate_width_samples=gate_width_t_s,
        fill_method=3)
    emg_timeseries.envelope(channel_idxs=[1])
    emg_timeseries.plot_full(
        channel_idxs=[1],
        axes=axes_emg_1,
        signal_type='env',
        baseline_bool=False,
        colors=[color])
axes_emg_1.set_xlabel('t (s)')
axes_emg_1.set_ylim([0, 3.0])
axes_emg_1.set_title('Gate-width effects')
axes_emg_1.legend(
    [str(gate_t) for gate_t in gate_width_t_ms],
    bbox_to_anchor=(0.01, 0.90, 0.98, 0.90),
    loc='lower left',
    ncols=5, 
    mode="expand", borderaxespad=0.)

peak_idxs = emg_timeseries.channels[1].peaks['ecg'].peak_df['peak_idx'].values
peak_idxs = peak_idxs[peak_idxs < 8 * fs_emg] - 10
t_emg_8 = np.array(t_emg)[np.array(t_emg) < 8 * fs_emg]

t_to_peak = (np.min(np.abs(
    repmat(peak_idxs/fs_emg, len(t_emg_8), 1) 
    - repmat(t_emg_8, len(peak_idxs), 1).T), axis=1))
axes_emg_1.fill_between(t_emg_8, 0.0, 3.0, where=t_to_peak < 1 / (2.5 * 2),
                color='black', alpha=0.1)
axes_emg_1.fill_between(t_emg_8, 0.0, 3.0, where=t_to_peak < 1*3 / (10 * 2),
                color='black', alpha=0.1)
axes_emg_1.fill_between(t_emg_8, 0.0, 3.0, where=t_to_peak < 1 / (5 * 2),
                color='black', alpha=0.1)
axes_emg_1.fill_between(t_emg_8, 0.0, 3.0, where=t_to_peak < 1 / (10 * 2),
                color='black', alpha=0.1)
axes_emg_1.fill_between(t_emg_8, 0.0, 3.0, where=t_to_peak < 1 / (20 * 2),
                color='black', alpha=0.1)


# Iterate over gating fill methods
methods = [0, 1, 2, 3]
colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red']

# colors = ['tab:purple', 'tab:blue', 'tab:cyan', 'tab:olive', 'tab:green',
#           'tab:red', 'tab:orange']

for _, (method, color) in enumerate(zip(methods, colors)):
    emg_timeseries.filter(
        lp_cf=500,
        hp_cf=25,
        order=3,
    )
    # emg_timeseries.filter(hp_cf=hp_cf, channel_idxs=[1], order=1)
    emg_timeseries.gating(channel_idxs=[1], fill_method=method)
    emg_timeseries.envelope(channel_idxs=[1])
    emg_timeseries.plot_full(
        channel_idxs=[1],
        axes=axes_emg_2,
        signal_type='env',
        baseline_bool=False,
        colors=[color])
axes_emg_2.set_xlabel('t (s)')
axes_emg_2.set_ylim([0, 3.0])
axes_emg_2.set_title('Fill methods')
methods_legend = ['Zero', 'Raw interp', 'Prior av', 'RMS interp']
axes_emg_2.legend(
    methods_legend,
    bbox_to_anchor=(0.01, 0.90, 0.98, 0.90),
    loc='lower left',
    ncols=5, 
    mode="expand", borderaxespad=0.)


# emg_timeseries.plot_full(
#     axes=axes_emg_2, signal_type='env', channel_idxs=[1], baseline_bool=False)


axes_emg_2.fill_between(t_emg_8, 0.0, 3.0, where=t_to_peak < 1 / (10 * 2),
                color='black', alpha=0.2)

# Ventilator data data
vent_timeseries.plot_full(axes_vent)
axis[-1].set_xlabel('t (s)')

# axis[-1, 0].axis('off')

axes_vent[-1].set_xlim([0, 8.0])

# 5. Envelope

In [ ]:
# Gate the EMG
emg_timeseries.filter(
        lp_cf=500,
        hp_cf=25,
        order=3,
    )
emg_timeseries.gating()


In [ ]:
# Compare envelope methods

# Side plot
fig = plt.figure(layout="constrained", figsize=(10, 6))

gs = GridSpec(6, 2, figure=fig)
axes_emg_1 = fig.add_subplot(gs[:3, 1])
axes_emg_2 = fig.add_subplot(gs[3:, 1], sharex=axes_emg_1)
for i in range(3):
    ax = fig.add_subplot(gs[i*2:i*2+2, 0], sharex=axes_emg_1)
axis = np.array(fig.axes)
axes_vent = axis[-3:]

# # EMG data
# # RMS windows
# rms_windows = [50, 100, 200, 250, 300]
# colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red', 'tab:orange']
# # colors = ['tab:purple', 'tab:blue', 'tab:cyan', 'tab:olive', 'tab:green',
# #           'tab:red', 'tab:orange']

# for _, (rms_window, color) in enumerate(zip(rms_windows, colors)):
#     emg_timeseries.envelope(
#         channel_idxs=[1], 
#         env_type='rms', 
#         env_window=(rms_window * fs_emg) // 1000)
#     emg_timeseries.plot_full(
#         channel_idxs=[1],
#         axes=axes_emg_1,
#         signal_type='env',
#         baseline_bool=False,
#         colors=[color])
# axes_emg_1.set_xlabel('t (s)')
# axes_emg_1.set_ylim([0, 3.0])
# axes_emg_1.set_title('RMS envelope, window length effect')
# axes_emg_1.legend(
#     [str(rms_window) for rms_window in rms_windows],
#     bbox_to_anchor=(0.01, 0.90, 0.98, 0.90),
#     loc='lower left',
#     ncols=5, 
#     mode="expand", borderaxespad=0.)


# # ARV windows
# arv_windows = [50, 100, 200, 250, 300]
# colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red', 'tab:orange']
# # colors = ['tab:purple', 'tab:blue', 'tab:cyan', 'tab:olive', 'tab:green',
# #           'tab:red', 'tab:orange']

# for _, (arv_window, color) in enumerate(zip(arv_windows, colors)):
#     emg_timeseries.envelope(
#         channel_idxs=[1], 
#         env_type='arv', 
#         env_window=(arv_window * fs_emg) // 1000)
#     emg_timeseries.plot_full(
#         channel_idxs=[1],
#         axes=axes_emg_1,
#         signal_type='env',
#         baseline_bool=False,
#         colors=[color])
# axes_emg_2.set_xlabel('t (s)')
# axes_emg_2.set_ylim([0, 3.0])
# axes_emg_2.set_title('ARV envelope, window length effect')
# axes_emg_2.legend(
#     [str(arv_window) for arv_window in arv_windows],
#     bbox_to_anchor=(0.01, 0.90, 0.98, 0.90),
#     loc='lower left',
#     ncols=5, 
#     mode="expand", borderaxespad=0.)

# Ventilator data data
vent_timeseries.plot_full(axes_vent)
axis[-1].set_xlabel('t (s)')

# axis[-1, 0].axis('off')

axes_vent[-1].set_xlim([0, 8.0])

In [19]:
# Calculate the envelope of the signal
emg_timeseries.envelope()
# Which equals:
# emg_timeseries.envelope(
#     env_window=None,
#     env_type='rms',
#     signal_type='clean',
# )
# Where:
# env_window:           Envelope window width, `None` defaults to fs // 5
# env_type:             'rms' for root-mean-square (default), 'arv' for average
#                       rectified
# signal_type:          Calculate the envelope over the clean data

# --> Data is stored in:
# emg_timeseries.channels[channel_idx].y_env

In [20]:
# Calculate the baseline for the EMG envelopes and p_vent
emg_timeseries.baseline()
# Which equals:
# emg_timeseries.baseline(
#     percentile=33,
#     window_s=int(7.5 * fs_emg),
#     step_s=fs_emg // 5,
#     method='default',
#     signal_type='env',
#     augm_percentile=25,
#     ma_window=None,
#     perc_window=None,
#     channel_idxs=[0, 1], 
# )
# Where:
# percentile:       Percentile of signal in the window to take as the baseline
#                   value 
# window_s:         Window length in samples (default: int(7.5 * fs))
# step_s:           Number of consecutive samples with the same baseline value
#                   (default: fs // 5)
# method:           'default' or 'slopesum'
# signal_type:      Calculate the baseline over the envelope (y_env) for emg 
#                   and over the original signal for p_vent
# augm_percentile   Augmented_percentile for slopesum baseline
# ma_window:        Moving average window in samples for average dy/dt in
#                   slopesum baseline
# perc_window:      'perc_window' for slopesum baseline.
# channel_idxs:     For all channels (None would default to this)

# --> Data is stored in:
# emg_timeseries.channels[channel_idx].y_baseline

vent_timeseries.baseline(
    channel_idxs=[0],
    signal_type='raw')

In [ ]:
# Plot the raw data with the envelope
# EMG data
fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(12, 6), sharex=True)
axes_emg = axis[:, 0]
colors = ['tab:cyan', 'tab:orange']
emg_timeseries.plot_full(
    axes=axes_emg, signal_type='clean', baseline_bool=False)
# Which equals:
# emg_timeseries.plot_full(
#     axes=axes_emg,
#     channel_idxs=[0, 1],
#     signal_type='clean',
#     colors=['tab:cyan', 'tab:orange'],
#     baseline_bool=False
# )
# Where:
# axes:             Matplotlib axes object
# channel_idxs:     For all channels (None would default to this)
# signal_type:      Plot clean emg
# colors:           Plot the signal, and baseline in the respective colors
# baseline_bool:    Plot the baseline

emg_timeseries.plot_full(axes=axes_emg, signal_type='env', colors=colors)

axes_emg[0].set_title('EMG data')
axes_emg[-1].set_xlabel('t (s)')

# Ventilator data data
axes_vent = axis[:, 1]
vent_timeseries.plot_full(axes_vent)
axes_vent[0].set_title('Ventilator data')
axes_vent[-1].set_xlabel('t (s)')

# axis[-1, 0].axis('off')

# axes_vent[-1].set_xlim([260, 295])

# 3. Identify Pocc peaks of interest in p_vent

In [40]:
# Find occlusion pressures
vent_timeseries.peep
vent_timeseries.find_occluded_breaths(
    vent_timeseries.p_vent_idx, start_idx=300*vent_timeseries.fs)
# Which equals:
# vent_timeseries.find_occluded_breaths(
#     pressure_idx=vent_timeseries.p_vent_idx,
#     peep=vent_timeseries.peep,
#     start_idx=360*vent_timeseries.fs,
#     end_idx=len(y_vent)-1,
#     prominence_factor=0.8,
#     min_width_s=vent_timeseries.fs//10,
#     distance_s=vent_timeseries.fs//2,
# )
# Where:
# pressure_idx:         Channel idx of ventilator pressure signal (auto-detected)
# peep:                 Ventilator PEEP level (auto-detected)
# start_idx             Look for Pocc from sample idx
# end_idx:              Look for Pocc up to sample idx
# prominence_factor     Factor setting minimum prominence of Pocc peak
# min_width_s:          Mnimum Pocc peak width in samples
# distance_s:           Mnimum Pocc peak distance in samples

# --> Peak data is stored in:
# vent_timeseries.channels[vent_timeseries.p_vent_idx].peaks['Pocc']

In [ ]:
# Detect on- and offsets of occlusion pressure peaks
p_vent = vent_timeseries.channels[vent_timeseries.p_vent_idx]
p_vent.peaks['Pocc'].detect_on_offset(baseline=p_vent.y_baseline)
# Which equals:
# p_vent.peaks['Pocc'].detect_on_offset(
#     baseline=p_vent.y_baseline,
#     method='default',   # == 'baseline_crossing'
#     fs=p_vent.fs,
#     slope_window_s=p_vent.fs // 5
# )
# Where:
# baseline:         Baseline for on-/offset detection.
# method:           On-/Offset detection method, 'default' == 'baseline_crossing'
# fs:               Signal fs
# slope_window_s:   For method 'slope_extrapolation', the window in samples
#                   where the slope is maximal

# The timings of the peaks is at:
print('The timings of the Pocc peaks is at:')
print(p_vent.peaks['Pocc'].peak_df['peak_idx']/p_vent.fs)

In [42]:
# Find supported breath pressures
v_vent = vent_timeseries.channels[vent_timeseries.v_vent_idx]
vent_timeseries.find_tidal_volume_peaks()
# Which equals:
# vent_timeseries.find_tidal_volume_peaks(
#     volume_idx=2,
#     start_idx=0,
#     end_idx=len(y_vent)-1,
#     width_s=v_vent.fs // 4,
#     threshold=0.25 * np.percentile(v_vent.y_raw, 90),
#     prominence=0.10 * np.percentile(v_vent.y_raw, 90),
#     threshold_new=0.5 * np.percentile(v_vent.y_raw[temp_peak_idxs], 90),
#     prominence_new=0.5 * np.percentile(v_vent.y_raw, 90),
#     pressure_idx=0,
# )
# Where:
# volume_idx:           Channel idx of ventilator volume signal (auto-detected)
# start_idx             Look for supported ventilator breath from sample idx
# end_idx:              Look for supported ventilator breath up to sample idx
# width_s:              Minimum supported ventilator breath width in samples
# threshold:            Minimum supported ventilator breath height in samples
# prominence:           Minimum supported ventilator breath prominence in samples
# threshold_new:        threshold, but filtering true breaths from originally identified peaks
# prominence_new:       prominence, but filtering true breaths from originally identified peaks
# pressure_idx:         Channel idx of ventilator pressure signal (auto-detected)

# --> Peak data is stored in:
# vent_timeseries.channels[vent_timeseries.p_vent_idx].peaks['ventilator_breaths']
# and
# vent_timeseries.channels[vent_timeseries.volume_idx].peaks['ventilator_breaths']

# 4. Identify all sEMG breaths, and find those closest to the Pocc peaks

In [43]:
# Find sEAdi peaks in one channel (sEAdi)
emg_di = emg_timeseries.channels[1]
emg_di.detect_emg_breaths(peak_set_name='breaths')

# Similar to above for Pocc and ventilator breaths:
emg_di.peaks['breaths'].detect_on_offset(
    baseline=emg_di.y_baseline
)
# --> Peak data is stored in:
# emg_di.peaks['breaths']

In [ ]:
# Find the EMG peaks with the peak timing closest to the Pocc timings, and
# link ventilator Pocc peaks to EMG breaths
t_pocc_peaks_vent = p_vent.peaks['Pocc'].peak_df['peak_idx'].to_numpy() / p_vent.fs

emg_di.link_peak_set(
    peak_set_name='breaths',
    t_reference_peaks=t_pocc_peaks_vent,
    linked_peak_set_name='Pocc',
)
# Where:
# peak_set_name:          Peakset in current signal (emg_di) to find closest peaks in
# t_reference_peaks:      Timing of the peaks in the other (Pocc) signal
# linked_peak_set_name:   Name of linked peakset in current signal. Defaults to
#                         peak_set_name + '_linked'

# --> Peak data is stored in:
# emg_di.peaks['Pocc']

print('The timings of the Pocc peaks is in sEMGdi are at:')
print(emg_di.peaks['Pocc'].peak_df['start_idx']/emg_di.fs)


In [ ]:
# Plot the identified Pocc peaks in p_vent and sEAdi
fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(10, 6), sharex=True)
axes_emg = axis[:-1, 0]
colors = ['tab:cyan', 'tab:orange', 'tab:red']
emg_timeseries.plot_full(axes_emg, signal_type='env')
emg_di.plot_markers(peak_set_name='Pocc', axes=axes_emg[1])
# Which equals:
# emg_di.plot_markers(
#     peak_set_name='Pocc',
#     axes=axes_emg[1],
#     valid_only=False,
#     colors=['tab:red', 'tab:red', 'tab:red'],
#     markers=['*', '*', '*'],
# )
# Where:
# peak_set_name:    Peakset to plot the markers from
# axes:             Axes object to plot in
# valid_only:       Plot only valid peaks
# colors:           List of colours for the peak, start and end marker
# markers:          List of markers for the peak, start and end marker


axes_emg[1].set_ylabel('sEAdi (uV)')
axes_emg[0].set_title('EMG data')
axes_emg[-1].set_xlabel('t (s)')

axes_vent = axis[:, 1]
vent_timeseries.plot_full(axes_vent)
p_vent.plot_markers(peak_set_name='Pocc', axes=axes_vent[0])
v_vent.plot_markers(peak_set_name='ventilator_breaths',
                    axes=axes_vent[2], colors='c')

axes_vent[0].set_title('Ventilator data')
axes_vent[-1].set_xlabel('t (s)')
axes_vent[-1].set_xlim([370, 410])

axis[-1, 0].axis('off')

In [ ]:
# Plot the individual peaks
n_peaks = len(emg_di.peaks['Pocc'].peak_df['start_idx'].to_numpy())
fig, axis = plt.subplots(nrows=2, ncols=n_peaks, figsize=(10, 6), sharey='row')

axes_emg = axis[0, :]
colors = ['tab:cyan', 'tab:orange', 'tab:red']
emg_di.plot_peaks(axes=axes_emg, peak_set_name='Pocc')
# Which equals:
# emg_di.plot_peaks(
#     peak_set_name='Pocc',
#     axes=axes_emg,
#     signal_type=emg_di.peaks['Pocc'].signal,    # == 'env'
#     margin_s=emg_di.fs // 2,
#     valid_only=False,
#     colors=['tab:blue', 'tab:orange'],
#     baseline_bool=True
# )
# Where:
# peak_set_name:    Peakset to plot the markers from
# axes:             Axes object to plot in
# signal_type:      Plot the envelope data, from which the peaks are determined
# margin_s:         Plot margin_s samples before start_idx and after end_idx 
# valid_only:       Plot only valid peaks
# colors:           List of colours for the peak [0] and baseline [1]
# baseline_bool:    Plot the baseline
    
emg_di.plot_markers(axes=axes_emg, peak_set_name='Pocc')
axes_emg[0].set_ylabel('sEAdi (uV)')

axes_vent = axis[1, :]
p_vent.plot_peaks(axes=axes_vent, peak_set_name='Pocc')
p_vent.plot_markers(axes=axes_vent, peak_set_name='Pocc')

for axis in axes_vent:
    axis.set_xlabel('t (s)')

# 5. Calculate features: ETP and PTP

In [ ]:
# Calculate PTPocc
p_vent.calculate_time_products(
    peak_set_name='Pocc', 
    aub_reference_signal=p_vent.y_baseline,
    parameter_name='PTPocc')
# Which equals:
# p_vent.calculate_time_products(
#     peak_set_name='Pocc', 
#     include_aub=True,
#     aub_window_s=5 * p_vent.fs,
#     aub_reference_signal=p_vent.y_baseline,
#     parameter_name='PTPocc')
# )
# Where:
# peak_set_name:        Peakset to calculate the timeproduct over
# include_aub:          Include the area under the baseline in the time product
# aub_window_s:         Window length in samples in which the local extreme is sought.
# aub_reference_signal: Optional reference signal to find the local extreme 
#                       in, else the signal underlying the peakset is taken.
# parameter_name:       Name of the parameter in peakset.peak_df

# --> Parameter data is stored in:
# p_vent.peaks['Pocc'].peak_df[parameter_name]

print(p_vent.peaks['Pocc'].peak_df)

In [ ]:
# Calculate ETPdi
emg_di.calculate_time_products(
    peak_set_name='Pocc', parameter_name='ETPdi')

print(emg_di.peaks['Pocc'].peak_df)

# 6. Test Pocc and sEMG quality of the peaks

In [ ]:
# Test Pocc quality
parameter_names = {
    'time_product': 'PTPocc'
}
p_vent.test_pocc_quality(
    peak_set_name='Pocc',
    parameter_names=parameter_names, 
    verbose=True)
# Which equals:
# p_vent.test_pocc_quality(
#     peak_set_name='Pocc',
#     cutoff={
#         'consecutive_poccs': 0,
#         'dP_up_10': 0.0,
#         'dP_up_90': 2.0,
#         'dP_up_90_norm': 0.8,
#     },
#     skip_tests=[],
#     parameter_names={
#         'time_product': 'PTPocc'
#     },
#     verbose=True
# )
# Where:
# peak_set_name:        Peakset to test
# cutoff:               The cut-off values for the tests
# skip_tests:           List of tests to skip. Any of: 'consecutive_poccs',
#                       'pocc_upslope'
# parameter_names:      Dictionary of renamed parameters. Required parameters
#                       are 'ventilator_breaths', and 'time_product', and need
#                       to be specified if renamed.
# verbose:              Print the output of the tests

# --> Peak quality data is stored in:
# p_vent.peaks['Pocc'].quality_values_df and
# p_vent.peaks['Pocc'].quality_outcomes_df

In [ ]:
# The peak-validity is updated in the peak_df:
print(p_vent.peaks['Pocc'].peak_df)

In [ ]:
# Test EMG quality
parameter_names = {
    'time_product': 'ETPdi'
}
emg_di.test_emg_quality(
    peak_set_name='Pocc', parameter_names=parameter_names)
# Which equals:
# emg_di.test_emg_quality(
#     peak_set_name='Pocc',
#     cutoff={
#         'interpeak_distance': 1.1,
#         'snr': 1.4,
#         'aub': 40,
#         'curve_fit': 30,
#         'aub_window_s': 5*emg_di.fs
#     },
#     skip_tests=[],
#     parameter_names={
#         'time_product': 'ETPdi'
#     },
#     verbose=True
# )
# Where:
# peak_set_name:        Peakset to test
# cutoff:               The cut-off values for the tests
# skip_tests:           List of tests to skip. Any of: 'interpeak_dist',
#                       'snr', 'aub', 'curve_fit'
# parameter_names:      Dictionary of renamed parameters. Required parameters
#                       are ['interpeak_distance', 'snr', 'aub', 'curve_fit'],
#                       and need to be specified if renamed.
# verbose:              Print the output of the tests

# --> Peak quality data is stored in:
# emg_di.peaks['Pocc'].quality_values_df and
# emg_di.peaks['Pocc'].quality_outcomes_df

In [ ]:
# The peak-validity is updated in the peak_df:
print(emg_di.peaks['Pocc'].peak_df)

In [ ]:
# Plot the individual peaks bell-fit
n_peaks = len(emg_di.peaks['Pocc'].peak_df['start_idx'].to_numpy())
fig, axis = plt.subplots(nrows=2, ncols=n_peaks, figsize=(10, 6), sharey='row')

axes_emg = axis[0, :]
colors = ['tab:cyan', 'tab:orange', 'tab:red']
emg_di.plot_peaks(axes=axes_emg, peak_set_name='Pocc')
emg_di.plot_markers(axes=axes_emg, peak_set_name='Pocc')
emg_di.plot_curve_fits(axes=axes_emg, peak_set_name='Pocc')
# Which equals:
# emg_di.plot_curve_fits(
#     peak_set_name='Pocc',
#     axes=axes_emg,
#     valid_only=False,
#     colors='tab:green'
# )
# Where:
# peak_set_name:    Peakset to plot the bell fit from
# axes:             Axes object to plot in
# valid_only:       Plot only valid peaks
# colors:           Colour for the bell-fit
axes_emg[0].set_ylabel('sEAdi (uV)')

axes_vent = axis[1, :]
p_vent.plot_peaks(axes=axes_vent, peak_set_name='Pocc')
p_vent.plot_markers(axes=axes_vent, peak_set_name='Pocc')

for axis in axes_vent:
    axis.set_xlabel('t (s)')

In [ ]:
# Plot the individual peaks bell-fit
n_peaks = len(emg_di.peaks['Pocc'].peak_df['start_idx'].to_numpy())
fig, axis = plt.subplots(nrows=2, ncols=n_peaks, figsize=(10, 6), sharey='row')

axes_emg = axis[0, :]
emg_di.plot_aub(axes=axes_emg, signal_type='env', peak_set_name='Pocc')
# Which equals:
# emg_di.plot_curve_fits(
#     peak_set_name='Pocc',
#     axes=axes_emg,
#     valid_only=False,
#     colors='tab:cyan'
# )
# Where:
# peak_set_name:    Peakset to plot the aub from
# axes:             Axes object to plot in
# signal_type:      Plot aub for this data, in this case the aub data
# valid_only:       Plot only valid peaks
# colors:           Colour for the aub
colors = ['tab:cyan', 'tab:orange', 'tab:red']
emg_di.plot_peaks(axes=axes_emg, peak_set_name='Pocc')
emg_di.plot_markers(axes=axes_emg, peak_set_name='Pocc')
axes_emg[0].set_ylabel('sEAdi (uV)')

axes_vent = axis[1, :]
p_vent.plot_peaks(axes=axes_vent, peak_set_name='Pocc')
p_vent.plot_markers(axes=axes_vent, peak_set_name='Pocc')

for axis in axes_vent:
    axis.set_xlabel('t (s)')